In [1]:
# do pip install with version ==
import geopandas as gpd 
import pandas as pd
import duckdb as dd

In [2]:
# get countis
counties_url = "https://gist.githubusercontent.com/sdwfrost/d1c73f91dd9d175998ed166eb216994a/raw/e89c35f308cee7e2e5a784e1d3afc5d449e9e4bb/counties.geojson"
df_counties = gpd.read_file(counties_url)

print(df_counties.head(5))


  STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID           NAME LSAD  \
0      06      075  00277302  0500000US06075  06075  San Francisco   06   
1      25      025  00606939  0500000US25025  25025        Suffolk   06   
2      31      007  00835826  0500000US31007  31007         Banner   06   
3      37      181  01008591  0500000US37181  37181          Vance   06   
4      48      421  01383996  0500000US48421  48421        Sherman   06   

        ALAND     AWATER                                           geometry  
0   121485107  479107241  POLYGON ((-122.51198 37.77113, -122.4654 37.80...  
1   150855462  160479920  POLYGON ((-71.19116 42.28306, -71.15689 42.330...  
2  1932676697     397069  POLYGON ((-104.05283 41.69795, -103.37039 41.6...  
3   653705784   42187365  POLYGON ((-78.49778 36.51448, -78.45728 36.541...  
4  2390651189     428754  POLYGON ((-102.16246 36.50033, -102.03234 36.5...  


In [3]:
# EIA power plant data
df_powerplants = gpd.read_file("https://services7.arcgis.com/FGr1D95XCGALKXqM/arcgis/rest/services/Power_Plants_Testing/FeatureServer/0/query?where=State%20%3D%20'WASHINGTON'&outFields=*&outSR=4326&f=json")
print(df_powerplants.head(5))

   OBJECTID  Plant_Code                       Plant_Name  Utility_ID  \
0     35906          99                     Frederickson       15500   
1     36055         371      Columbia Generating Station       20160   
2     36178         550  Kettle Falls Generating Station       20169   
3     36204         607                         Fredonia       15500   
4     36212         622                  South Fork Tolt       16868   

             Utility_Name       sector_name               Street_Address  \
0  Puget Sound Energy Inc  Electric Utility           4714 192nd St East   
1        Energy Northwest  Electric Utility        76 N Power Plant Loop   
2             Avista Corp  Electric Utility             1151 Hyw. 395 N.   
3  Puget Sound Energy Inc  Electric Utility              13085 Ball Road   
4  City of Seattle - (WA)  Electric Utility  20030 Cedar Falls Road S.E.   

           City   County       State  ...  Nuclear_MW Crude_MW Solar_MW  \
0        Tacoma   Pierce  Washingto

In [4]:
# Overture 
db = dd.connect()
db.execute("INSTALL spatial")
db.execute("INSTALL httpfs")
db.execute("""
LOAD spatial;
LOAD httpfs;
SET s3_region='us-west-2';
""")

db.sql("SET azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=overturemapswestus2;AccountKey=;EndpointSuffix=core.windows.net';")
db.sql("""
COPY(
  SELECT
    id,
    names.primary as primary_name,
    subtype,
    geometry
  FROM read_parquet('azure://release/2024-09-18.0/theme=base/type=infrastructure/*', filename=true, hive_partitioning=1)
  WHERE names.primary IS NOT NULL AND subtype = 'power'
  AND bbox.xmin BETWEEN -124.763068 AND 45.54383071539715
  AND bbox.ymin BETWEEN -116.9161607504075 AND 49.00240502974029
) TO 'washington.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');
""")

In [ ]:
gdf = gpd.read_file('washington.geojson')

print(len(gdf)
